# Earnings Streak 

From "Streaks in Earnings Surprises and the Cross-Section of Stock Returns" (Loh & Warachka, 2012)

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pandas_datareader as pdr
import quantstats as qs

from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
pathDataIntermediate = '~/Mirror/Work/research/main/investing/equities_investing/anomalies_research/replications/yilin_resources/yilin_data'

In [3]:
# Convert to first of next month to eventually merge with RDQ data 
def first_day_of_next_month(date):
    next_month = date.replace(day=1) + pd.DateOffset(months=1)
    return next_month

***
## Setup

In [4]:
# LOAD DATA
IBES_EPS_Adj = pd.read_csv(pathDataIntermediate + '/IBES_EPS_Adj.csv')
IBES_EPS_Adj['tickerIBES'].fillna('NA', inplace=True)
IBES_EPS_Adj = IBES_EPS_Adj.loc[IBES_EPS_Adj['fpi']==6]
IBES_EPS_Adj.dropna(subset=['actual', 'meanest','price'], inplace=True)
IBES_EPS_Adj = IBES_EPS_Adj[IBES_EPS_Adj['price'] > 5.00]

In [5]:
# Dataframe with Earnings Value
# Tickers IBES weird identifier of unknown link to CRSPCOMP
IBES_EPS_Adj.columns

Index(['Unnamed: 0', 'fpi', 'tickerIBES', 'statpers', 'fpedats', 'anndats_act',
       'meanest', 'actual', 'medest', 'stdev', 'numest', 'prdays', 'price',
       'shout', 'time_avail_m'],
      dtype='object')

In [6]:
# Actual release date as date of availability
# Add one to avoid look-ahead bias (this creates availability only at the beginning of next month)
# This works in the context of monthly re-allocation 
IBES_EPS_Adj['time_avail_m'] = pd.to_datetime(IBES_EPS_Adj['anndats_act']).dt.to_period('M') + 1

In [7]:
# Keep the last forecast before the actual release to compute surprise
IBES_EPS_Adj = IBES_EPS_Adj.sort_values(['tickerIBES', 'time_avail_m','anndats_act','statpers']).reset_index(drop=True)
IBES_EPS_Adj = IBES_EPS_Adj.sort_values(['tickerIBES', 'time_avail_m']).drop_duplicates(['tickerIBES', 'time_avail_m'], keep='last').reset_index(drop=True)

In [8]:
# Define earnings surprise (positive / negative surprise) and Streak (consistent Surp)
IBES_EPS_Adj['earnings_surprise'] = (IBES_EPS_Adj['actual'] - IBES_EPS_Adj['meanest'])/ IBES_EPS_Adj['price']

# Lagged earnings surprise
IBES_EPS_Adj['past_earnings_surprise'] = IBES_EPS_Adj.groupby(['tickerIBES'])['earnings_surprise'].shift(1)

# Defining streak with 0 values
IBES_EPS_Adj['positive_earnings_streak'] = 0
IBES_EPS_Adj['negative_earnings_streak'] = 0

# If sign of streak same as previous surprise, define a streak
# If sign negative, define negative streak, otherwise it's a positive streak
IBES_EPS_Adj['negative_earnings_streak'].loc[(np.sign(IBES_EPS_Adj['earnings_surprise']) == -1) & (np.sign(IBES_EPS_Adj['earnings_surprise']) == np.sign(IBES_EPS_Adj['past_earnings_surprise']))] = 1
IBES_EPS_Adj['positive_earnings_streak'].loc[(np.sign(IBES_EPS_Adj['earnings_surprise']) == 1) & (np.sign(IBES_EPS_Adj['earnings_surprise']) == np.sign(IBES_EPS_Adj['past_earnings_surprise']))] = 1

In [25]:
# Use the Signal Master to link permno and IBES ticker 
SignalMasterTable = pd.read_csv(pathDataIntermediate + '/SignalMasterTable.csv',
                                usecols = ['permno','time_avail_m','tickerIBES'])

In [26]:
# Convert to monthly
SignalMasterTable['time_avail_m'] = pd.to_datetime(SignalMasterTable['time_avail_m']).dt.to_period('M')
SignalMasterTable['tickerIBES'].loc[SignalMasterTable['permno']==81191] = 'NA'

# Merge many to one to grab the returns 
df = pd.merge(SignalMasterTable, IBES_EPS_Adj, on=['time_avail_m','tickerIBES'], validate = 'm:1', how='left')

# Drop useless columns
df.drop(columns=['fpi','tickerIBES'], inplace=True)

In [27]:
crsp_m = pd.read_csv('../../yilin_resources/yilin_data/mCRSP.csv', parse_dates=['date'])
crsp_m['date'] = crsp_m['date'].dt.to_period('M')
crsp_m = crsp_m.rename(columns={'date':'time_avail_m'})
df = pd.merge(df, crsp_m[['time_avail_m', 'permno', 'ret']], on=['time_avail_m','permno'], how='left')

In [28]:
# def define_universe(df, file):
#     # Load the list of permno values from the text file
#     with open(file, 'r') as file:
#         permno_list = [int(line.strip()) for line in file]

#     # Filter the DataFrame based on permno values
#     universe = df[df['permno'].isin(permno_list)]
#     return universe

# df = define_universe(df, '../resources/sp500_permnos.txt')

In [29]:
# Drop stale or empty
df['anndats_act'] = df.groupby(['permno'])['anndats_act'].ffill()
df.dropna(subset=['anndats_act'], inplace=True)
df['time_avail_m'] = df['time_avail_m'].apply(lambda x: x.to_timestamp())
#df = df.drop(df[(df['time_avail_m'] - pd.to_datetime(df['anndats_act']))/np.timedelta64(1,'M')>6].index)

In [30]:
df['log_ret'] = np.log(1 + df['ret'])

df['cumret'] = df.groupby('permno')['log_ret'].shift(-1) + df.groupby('permno')['log_ret'].shift(-2) + df.groupby('permno')['log_ret'].shift(-3) + df.groupby('permno')['log_ret'].shift(-4) + df.groupby('permno')['log_ret'].shift(-5) + df.groupby('permno')['log_ret'].shift(-6)

In [31]:
df

,permno,time_avail_m,Unnamed: 0,statpers,fpedats,anndats_act,meanest,actual,medest,stdev,...,prdays,price,shout,earnings_surprise,past_earnings_surprise,positive_earnings_streak,negative_earnings_streak,ret,log_ret,cumret
141,10001.0,1996-05-01,5025529.0,1996-04-18,1995-12-31,1996-04-18,0.23,0.1933,0.23,NaN,...,1996-04-17,5.75,3.420,-0.006383,NaN,0.0,0.0,-0.021277,-0.021506,0.039357
142,10001.0,1996-06-01,NaN,NaN,NaN,1996-04-18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.060290,-0.062184,0.040273
143,10001.0,1996-07-01,NaN,NaN,NaN,1996-04-18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023438,0.023167,0.076825
144,10001.0,1996-08-01,NaN,NaN,NaN,1996-04-18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.038168,0.037458,0.039368
145,10001.0,1996-09-01,NaN,NaN,NaN,1996-04-18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041765,0.040916,0.010552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3879037,93429.0,2018-04-01,NaN,NaN,NaN,2018-02-09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.064154,-0.066305,NaN
3879038,93429.0,2018-05-01,NaN,NaN,NaN,2018-02-09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.083817,-0.087539,NaN
3879039,93429.0,2018-06-01,2209645.0,2018-04-19,2018-03-31,2018-05-04,1.26,1.3800,1.27,0.04,...,2018-04-18,109.33,112.666,0.001098,0.0,0.0,0.0,0.066728,0.064596,NaN
3879040,93429.0,2018-07-01,NaN,NaN,NaN,2018-05-04,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.066686,-0.069013,NaN


In [32]:
# df['EarningsStreak'] = df['surp'].copy()
# df['EarningsStreak'] = df.groupby(['permno'])['EarningsStreak'].ffill()

# df['EarningsStreak'].replace([np.inf, -np.inf], np.nan,inplace=True)
# df.dropna(subset=['EarningsStreak'], inplace=True)
# df = df.reset_index(drop=True)
df['time_avail_m'] = df['time_avail_m'].dt.to_period('M')
df.rename(columns={'time_avail_m': 'yyyymm'}, inplace = True)
df = df[['permno','yyyymm', 'cumret','positive_earnings_streak', 'negative_earnings_streak']]

***
## Portfolio and Analysis

In [33]:
# Long and short portfolio
L_trades = df[df['positive_earnings_streak'] == 1]
S_trades = df[df['negative_earnings_streak'] == 1]

L = L_trades.groupby('yyyymm')['cumret'].mean().asfreq('M', fill_value=0)
S = S_trades.groupby('yyyymm')['cumret'].mean().asfreq('M', fill_value=0)

LS = (L - S).dropna()

LS.index = LS.index.to_timestamp()

start_date = LS.index.min()
end_date = LS.index.max()

In [34]:
# Benchmarking with the FF5
FF_five_with_rf= pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start='1900')[0]
FF_five = FF_five_with_rf.drop(columns={'RF'}) / 100
FF_five.index = FF_five.index.to_timestamp()
FF_five = FF_five.loc[(FF_five.index >= start_date) & (FF_five.index <= end_date)]

In [35]:
# Benchmarking with SP500
sp500_rets = pd.read_csv('../../resources/sp500_rets.csv')
sp500_rets['MthCalDt'] = pd.to_datetime(sp500_rets['MthCalDt'])
sp500_rets['MthCalDt'] = sp500_rets['MthCalDt'].apply(first_day_of_next_month)
sp500_rets = sp500_rets.set_index('MthCalDt')
sp500_rets = sp500_rets.loc[(sp500_rets.index >= start_date) & (sp500_rets.index <= end_date)]

In [36]:
print(sm.OLS(LS, np.ones_like(LS)).fit(cov_type='HAC', cov_kwds={'maxlags': 10}).summary())

                            OLS Regression Results                            
Dep. Variable:                 cumret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 28 Aug 2023   Prob (F-statistic):                nan
Time:                        23:39:52   Log-Likelihood:                 414.47
No. Observations:                 441   AIC:                            -826.9
Df Residuals:                     440   BIC:                            -822.8
Df Model:                           0                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0538      0.008      7.161      0.0

In [37]:
print(sm.OLS(LS, sm.add_constant(FF_five)).fit(cov_type='HAC', cov_kwds={'maxlags': 10}).summary())

                            OLS Regression Results                            
Dep. Variable:                 cumret   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.020
Date:                Mon, 28 Aug 2023   Prob (F-statistic):              0.405
Time:                        23:39:52   Log-Likelihood:                 417.24
No. Observations:                 441   AIC:                            -822.5
Df Residuals:                     435   BIC:                            -797.9
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0560      0.007      7.571      0.0

In [38]:
print(sm.OLS(LS, sm.add_constant(sp500_rets)).fit(cov_type='HAC', cov_kwds={'maxlags': 10}).summary())

                            OLS Regression Results                            
Dep. Variable:                 cumret   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.5972
Date:                Mon, 28 Aug 2023   Prob (F-statistic):              0.440
Time:                        23:39:52   Log-Likelihood:                 414.86
No. Observations:                 441   AIC:                            -825.7
Df Residuals:                     439   BIC:                            -817.6
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
INDNO       5.476e-08   8.01e-09      6.840      0.0

In [39]:
# qs.reports.html(LS, benchmark=sp500_rets, rf=0.04, title='Earnings Streak Tearsheet', download_filename='../results/es_tearsheets.html')